In [3]:
from pathlib import Path
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as clr
cmap_veg = clr.LinearSegmentedColormap.from_list('veg', ['#ffffe5','#f7fcb9','#d9f0a3','#addd8e','#78c679','#41ab5d','#238443','#006837','#004529'])



In [9]:
basepath = Path("/workspace/data/s3/earthnet/earthnet2023/")
train_paths = list(basepath.glob("*/*.nc"))
test_paths = list(basepath.glob("test/*/*.nc"))
pred_paths = list(Path("/workspace/crobin/earthnet-models-pytorch/experiments/en23/convlstm_ae/convlstm_ae/config_anomalie/preds/iid/").glob("*/*"))

In [10]:
pred_path = pred_paths[6]
name = str(pred_path)[-12:]
test_path = list(basepath.glob("test/*/" + name))[0]

In [16]:
# Select the days with available data
minicube = xr.open_dataset(test_path)
s2_avail = np.array(minicube.s2_avail.values)
print(s2_avail)
# s2 is every 5 days
t_len = s2_avail.shape[0]
# time = [i for i in range(4, t_len, 5)]
time = np.where(minicube.s2_avail.values == 1)[0]


[nan nan nan ... nan nan nan]
2191
[   7   17   27   37   47   67   77   87   97  107  127  137  147  157
  167  177  187  197  207  217  227  237  247  257  267  277  287  297
  307  317  327  337  347  357  367  397  407  417  427  437  447  457
  467  477  487  497  507  517  527  537  547  552  557  562  567  572
  577  582  587  592  597  602  607  612  617  627  632  642  652  657
  662  667  677  682  687  692  697  707  712  717  722  727  732  737
  742  747  752  757  762  767  772  777  787  792  802  807  812  817
  822  827  832  837  842  847  852  857  862  867  872  877  882  887
  892  897  902  907  912  917  922  927  932  937  942  947  952  957
  962  967  972  977  982  992  997 1002 1007 1012 1017 1022 1027 1032
 1037 1042 1047 1052 1057 1062 1067 1072 1077 1082 1087 1092 1097 1102
 1107 1112 1117 1122 1127 1132 1137 1142 1147 1152 1157 1162 1167 1172
 1177 1182 1187 1192 1197 1202 1207 1212 1217 1222 1227 1232 1237 1242
 1247 1252 1257 1262 1272 1277 1282 1287 1